In [2]:
import pandas as pd
import os
import re
import sys

#os.chdir('../')
sys.path.insert(0, '/Users/alanliu99/Documents/GitHub/yelp-recommender-system/src')
from tfidf import *

In [3]:
os.chdir('../')
business = pd.read_csv('data/business.csv')
#review = pd.read_csv('test/testdata/test_review.csv')
las_vagas_reviews = pd.read_csv('data/Las_Vegas_review.csv')

In [5]:
las_vagas_reviews

,name,business_id,city,categories,review_id,text,stars,user_id
0,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",WUAt5-krh075Ie44np8Hew,So why would I be giving a Fast Food chain loc...,5.0,YDkC5VVT8s9NlIZtAr8NUA
1,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",S9vIs-je49jqbhpWSQ8yHg,I come here bout 3x's a mo. & I just can't do ...,2.0,mxtalrN7VHVQwGjv8ln5yg
2,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",i2Soid_X8XsSTyIrPOst6g,WORST experience EVER!!!!! never have i ate an...,1.0,xammeA3ftpFoAN13v5Blyg
3,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",uDYGaAOUgo7SLuH5nUZs3w,Hot fresh food usually. Staff seems to turn o...,4.0,gQhFacMGI41mfVXqw1E7bQ
4,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",Xg7HuBvuZHTBog1G_2Wk0w,Beer battered cod fish seems really rubbery. I...,3.0,0PXhJMztE3ijzQPjiwPERg
...,...,...,...,...,...,...,...,...
1503839,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",3TD_bRi98t5UDLuesY66Hw,Came to Vegas with the fam and some other fami...,1.0,UjBiVw0WFmEbgK6qgap3QA
1503840,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",IeowybrKAmmf9njf7pa1rA,So there are alot of pretty bad reviews but I ...,3.0,F5YcdVMqhDAfhGZsMCXDTA
1503841,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",r7maiBTmlfFRcaLAmwY0Aw,All the buffets are pretty much the same. Ove...,2.0,CPLndzPfGfJzjqjzXCr9_g
1503842,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",UPn6fAXxo7SEaQL4xkUgFA,Don't come here for dinner. You can see the b...,1.0,pSoe4XBftnJiTgG1fPw52g


In [ ]:
cities = ['Las Vegas','Phoenix','Toronto']
business.dropna(inplace = True)
restaurants_business = business[business.categories.str.contains("Restaurants")]
business_city = restaurants_business[restaurants_business.city == 'Las Vegas']
bus_rev_city = business_city.merge(reviews, on = 'business_id')
bus_rev_city = bus_rev_city.drop(columns = ['Unnamed: 0'])

In [ ]:
larger_than_5_review_user = las_vegas_all_reviews[las_vegas_all_reviews.user_id.isin(filtered_larger_5.index.values)]
larger_than_5_review_user

In [ ]:
larger_than_5_review_user.to_csv('5_more_review_Las_Vegas.csv', index = False)

### Preprocess for TF-IDF

In [15]:
def preprocess_group_user_review(review_df):
    user_df = review_df[['text','categories','user_id']].groupby('user_id').agg({'text':list,
                                                                            'categories':lambda column: ' '.join(column)}
                                                                          ).reset_index()
    user_df.text = user_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return user_df

In [6]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                          'stars':np.mean}
                                                                                        ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return restaurant_df.merge(business_df[['name', 'business_id']], on = 'business_id')

In [17]:
joined_user = preprocess_group_user_review(las_vagas_reviews)
joined_user

,user_id,text,categories
0,---1lKK3aKOuomHnwAkAow,I like the Lamb Tandoori dish they do a lot ...,"Pakistani, Restaurants, Indian Local Flavor, R..."
1,---RfKzBwQ8t3wu-LXvx3w,This pizza rules Great people working here...,"Italian, Restaurants, Pizza"
2,---udAKDsn0yQXmzbWQNSw,We were visiting from SoCal and was in the m...,"Restaurants, Filipino Food, Restaurants, Thai"
3,--0OjK-YhWtnTo4zO9llqw,The hotel is old and not well maintained W...,"Beauty & Spas, Hotels, Restaurants, Lounges, R..."
4,--0sXNBv6IizZXuV-nl0Aw,This place is awesome The pasta is homemade...,"Noodles, Restaurants, Italian"
...,...,...,...
536316,zzyLd6p8pcNdkd4Hrv08bw,Pretty decent Thai food in Vegas Super busy...,"Automotive, Wine Bars, Buffets, Car Dealers, B..."
536317,zzyzP--p98zZtx359NvgGA,I love middle eastern food Love their chick...,"Mediterranean, Restaurants, Middle Eastern"
536318,zzzPVqSxSvjzlLR3Q7wsUw,The best Japanese food I have ever eaten Yo...,"Latin American, Restaurants, Japanese, Sushi B..."
536319,zzzTkKLFo9CaeZnfO4TvzA,Have you ever had to eat trash and have to p...,"Restaurants, Asian Fusion, Mexican, Chinese"


In [8]:
joined_restaurant = preprocess_group_restaurant_review(las_vagas_reviews, business)
joined_restaurant

,business_id,text,stars,name
0,--9e1ONYQuAa-CB_Rrw7Tw,On yelp 5 stars Woohoo as good as it gets...,4.117841,Delmonico Steakhouse
1,--q7kSBRb0vWC8lSkXFByA,This is a fun spot to watch games and eat go...,4.000000,Double Play Sports Bar
2,-0RkJ_uIduNLWQrphbADRw,Amazing food and service So grateful for th...,4.132653,Rooster Boy Cafe
3,-153AjTW5luZPK4omEujWA,Eisenhower once said Some people wanted ch...,3.000000,Jody Maroni's Sausage Kingdom
4,-1m9o3vGRA8IBPNvNqKLmA,Nice restaurant in Park MGM Las Vegas But I...,4.435388,Bavette's Steakhouse & Bar
...,...,...,...,...
6926,zwNC-Ow4eIMan2__bS9-rg,I can t complain My new favorite Mexican f...,4.052067,Taco Y Taco
6927,zwddv6HjTjDx9fOr3MFtwQ,Located on the 5th floor of the Imperial Pal...,2.461538,Quesadilla
6928,zwmKWhyyV-zejEUbQzBtHw,We heard of the XLBs here from Tony s review...,4.333333,A & K Chinese Restaurant
6929,zx_j6OuuHHa2afVoAZuLpA,My friend lives in Las Vegas but I m from L...,4.532258,what's Crepe


## Filter out the grouped dataframe (optional)

In [9]:
filtered_joined_restaurant = joined_restaurant[joined_restaurant.stars > 3]
filtered_joined_restaurant 

,business_id,text,stars,name
0,--9e1ONYQuAa-CB_Rrw7Tw,On yelp 5 stars Woohoo as good as it gets...,4.117841,Delmonico Steakhouse
1,--q7kSBRb0vWC8lSkXFByA,This is a fun spot to watch games and eat go...,4.000000,Double Play Sports Bar
2,-0RkJ_uIduNLWQrphbADRw,Amazing food and service So grateful for th...,4.132653,Rooster Boy Cafe
4,-1m9o3vGRA8IBPNvNqKLmA,Nice restaurant in Park MGM Las Vegas But I...,4.435388,Bavette's Steakhouse & Bar
5,-1vfRrlnNnNJ5boOVghMPA,What makes this a 5 star all you can eat sus...,3.261905,Red Ginseng Narita Sushi & BBQ
...,...,...,...,...
6925,zuwba6QEBIDZT0tJZmNhdQ,Amazing place to eat chill environment and s...,3.690909,Kobe Sushi Bar
6926,zwNC-Ow4eIMan2__bS9-rg,I can t complain My new favorite Mexican f...,4.052067,Taco Y Taco
6928,zwmKWhyyV-zejEUbQzBtHw,We heard of the XLBs here from Tony s review...,4.333333,A & K Chinese Restaurant
6929,zx_j6OuuHHa2afVoAZuLpA,My friend lives in Las Vegas but I m from L...,4.532258,what's Crepe


In [ ]:
filtered_joined_restaurant = joined_restaurant[joined_restaurant.stars > 3]
filtered_joined_restaurant 

### Convert the text review dataframe into the TF-IDF featue matrix

In [11]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.1, stop_words='english')
tfidf_matrix = tf.fit_transform(filtered_joined_restaurant.text.apply(lambda x: x.lower()))

### User - User Simple TF-IDF and Cosine Similarity recommendation

In [ ]:
def user_recommendations(ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        User ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).head(k)

In [12]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
user_user_df = pd.DataFrame(cosine_sim, columns=t.user_id, index=t.user_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
user_id = '---1lKK3aKOuomHnwAkAow' # input user id
add_on = t[['user_id','text', 'categories']]

user_recommendations(user_id, user_user_df, add_on, k=5)

### Resraurant - Reataurant TF-IDF approach

In [26]:
def restaurant_recommendations(business_ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        Restaurant Business ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,business_ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(business_ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).drop_duplicates().head(k)

In [13]:
# direct calculate the cosine similarity and generate the dataframe
# cosine_sim = cosine_similarity(tfidf_matrix)
rest_rest_df = pd.DataFrame(cosine_sim, columns=filtered_joined_restaurant.business_id, index=filtered_joined_restaurant.business_id)

In [30]:
# given a user ID and the similarity matrix, recommend a list of similar user
business_id = '4Nj2ktP2NjCbv2aKeH3ZBg' # input business id
add_on = las_vagas_reviews[['business_id','categories','name']].merge(filtered_joined_restaurant[['business_id','stars']])

restaurant_recommendations(business_id, rest_rest_df, add_on, k=15)

,business_id,categories,name,stars
0,UMqfeCItzQ2glr4d9apGlA,"Coffee Roasteries, Restaurants, Bakeries, Tea ...",Sunrise Coffee,4.483969
1310,rq5dgoksPHkJwJNQKlGQ7w,"Cafes, Breakfast & Brunch, Restaurants, Americ...",Sambalatte,4.030430
2263,x1bRIbodhd1AmYa0UAJwZw,"Coffee Roasteries, Food, Restaurants, Breakfas...",Bad Owl Coffee Roasters,4.064024
2591,VWtT4DKxQ4YVRWT88Z5SMA,"Cafes, Coffee & Tea, Food, Restaurants",Sambalatte,3.817460
2969,wZJOlSwUZcVQuLt9sZeV2A,"Bakeries, Restaurants, Coffee & Tea, Juice Bar...",The Cuppa,4.203791
3180,kDCyqlYcstqnoqnfBRS5Og,"Caribbean, Coffee & Tea, Sandwiches, Desserts,...",Pour Coffeehouse,4.605150
3413,ybHlmdUHLPKfv85bRK4Wtw,"Food, Coffee & Tea, Bagels, Cafes, Breakfast &...",Grouchy John's Coffee,4.621622
4671,x71EvWJDVKZfM-OiYDWZ6A,"Arts & Entertainment, Food, Coffee & Tea, Brea...",Perk Up Coffee Shop,4.059701
4872,KdooPJ0ePho3p49WA3JpQw,"Food, Coffee & Tea, Italian, Cafes, Sandwiches...",Roma Coffee,4.360000
4997,CoyeXg8FBsS_d20QzNIy-A,"Cafes, Latin American, Breakfast & Brunch, Cof...",Makers & Finders,4.205929


## Generate Top 100 nearest restaurant for each resraurant in Las Vegas

In [66]:
def generate_n_dataframe(df, n = 20):
    column = ['business_id']
    for i in range(n):
        column_name = 'Top ' + str(i)
        column.append(column_name)
    # top N recommended restaurant dataframe for all restaurants
    top_df = pd.DataFrame(columns = column)
    # iterate through all restaurants in the dataframe
    for i in df.index:
        ix = df.loc[:,i].to_numpy().argpartition(range(-1,-n,-1))
        high_recomm = df.columns[ix[-1:-(n+2):-1]]
        top_df = top_df.append(pd.Series(high_recomm.values, index = column),ignore_index=True)
    return top_df

In [68]:
result_df = generate_n_dataframe(rest_rest_df, n = 100)
result_df

,business_id,Top 0,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,...,Top 90,Top 91,Top 92,Top 93,Top 94,Top 95,Top 96,Top 97,Top 98,Top 99
0,--9e1ONYQuAa-CB_Rrw7Tw,qmIHO-6T_KEfPC9jyGDamQ,xkVMIk_Vqh17f48ZQ_6b0w,gdWy-x5KbCbMrwIwI87mMg,6xgcHeLad-VaoTIQewK84A,16Fplxu-OwVmTEFxQAUP4g,J4CATH00YZrq8Bne2S4_cw,MpovsyjkSkchD7ssAdEVKQ,07AZL5XenCQ_-op_onKLdw,7fxebHYUwIF6CakxSr70iQ,...,Yiz2cRBakfVUOHsThLKyoQ,bfp51Aaft7T3M9gQnVsI_g,5EIuCcoHY2evAhFMBV6G6g,nxzqOt8sABsgf53c4CgMqg,bK_eTLw_uEuEyMzdZlKN6A,ZtsnwjxjzMUI6tDZNrJUeA,d482l58ho0_IMdFkXX1F5Q,pmoIMK8zGWvKsVOPDBYfzg,7qFtiPg0gdbk911_mj6LoA,qA4MWdUmaXYkrZ223cFfFA
1,--q7kSBRb0vWC8lSkXFByA,DC3Uyai-vCCUgMmMvLESlw,X11JfeIVH9d6qMrDqA-N7A,W_qjGgy4LsEQnKKjIscDNg,GU0zNpgisY-pV3U2Sfdp6A,lLMJsTchKYF1O0fpNWpCRA,8wt-saX_qXaqVCXQHc5qXw,KPV_FVNWkgmYh1ArVlt6kg,mcWAIj1u1S-QDBhB7CiIdQ,nO2mE8gLNxi1Fsm6bZsn-g,...,khXRwuslAgvZ8PkqCgiBKw,9IpKGHaPy3hhXo9g5Bt7lg,qEg0K_45TBlzHMOR9GC4Pg,5gv6AqHXfi3gJV4fb432Vw,8pmGz3Lg4f_r7HdDZPa4dQ,d3zW_b-JWlxMneNn7K9xew,zY6f_cFY8HTI3hLlJj3SOg,aVmuRwieNxpW8vfC3s1jpg,ai8nfTBNvL579cjIuqZajQ,BORPULADzZV6_LJZzcsywQ
2,-0RkJ_uIduNLWQrphbADRw,2SfSzEd3B7WimeZac23zhg,R_fQGSUQv0L_fRiuua47dw,WETHRoXB2IbixndSlOktJA,0EgYXYjt2XJL4hlsKnzrcw,8fFTJBh0RB2EKG53ibiBKw,OYHpqY6-3C-wXR-uPbHhwA,xnVkYE3iMp_aZniiCIuD0g,8chcaZIRlP4p5anpzUB3FQ,v1AE1LLWDOqe8ubOzqKADg,...,9YF-nxh6xDUvMoOGWSDLkg,G58YATMKnn-M-RUDWg3lxw,a0v7Si0DK4cIko7AQY4YXg,REM8BezFuK4MQ6j2k1I7-Q,yjME88ri8wpBDm6i0xoi9Q,Ipkl7KtlqxCe8bMMkwHwPQ,8RFT1woy0WnTp62ndVJgow,_sh6mIBWZis66mAjkjN8Qg,jLxeBgWhLRbII2ACkgH1Sg,klddGAcNIpE_aB5sd1hNXw
3,-1m9o3vGRA8IBPNvNqKLmA,gdWy-x5KbCbMrwIwI87mMg,qmIHO-6T_KEfPC9jyGDamQ,bpRo8L8dkhgbJhdIKa9mwA,6xgcHeLad-VaoTIQewK84A,--9e1ONYQuAa-CB_Rrw7Tw,Fy4t2XHIRKwMk_ocCZze0Q,ubz4CaZXagQuGv2N9gFAdw,n1Ng1q300iLHYZELtOd5FA,sB1lPLICz4nrOuuqgLv50A,...,SAIrNOB4PtDA4gziNCucwg,PDIh1lVtKlUqPJROXYJQvQ,7qFtiPg0gdbk911_mj6LoA,nxzqOt8sABsgf53c4CgMqg,ZXfMiPsPv-cH7GMQ-5-mhA,r0LgymS8idM1K2exNl-CGg,ZtsnwjxjzMUI6tDZNrJUeA,gc_GGat7TgNCmrgc4nVOgw,61qDFkNrHKzyA9n3dlgcEw,BjH8Xepc10i6OhCDQdX6og
4,-1vfRrlnNnNJ5boOVghMPA,dZB5VuI4mCVRz8qQUwUgCg,MXC9pwIxovWUc9yu1F8OxA,xfzDtGxx6B4a27RAyah69w,BjrKNWhtQkedHw8hP_0Bjg,qpub9UXk8oBJr95oq-MbIg,HQcel0uvg7N7KfhHJChfSQ,HfuIfYB_CxMAAJiEnxDl9w,qFZ7ReYNpjHuvSxOqXqHMg,5JjGTaZkHbbjBrzBODcsAQ,...,q3IEuy9YPsHh0hpqX1HJZw,tjYHsz4ydS6GuBSv-uifQA,-RJ216TTIghZshCkUlD1WQ,bixMVkra8aJ1zcx026N8Bw,TFybLreUjQyx0zLq5tW7VA,T_YWS3Rp7-iu69VS5jcQPg,gqqHdBJqOVHDMy3jAdSkmw,tgFQQ4AK42geNLEMEioPeQ,LEzaRCdhDfO2eM75HZNnUQ,O7ot_LMlCfLpOP9tBqeNfw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,zuwba6QEBIDZT0tJZmNhdQ,M-SS_Kr0zlC7v8kBSRa74g,HfuIfYB_CxMAAJiEnxDl9w,WrHu03srbhJAw6bnTIixdw,dZB5VuI4mCVRz8qQUwUgCg,5JjGTaZkHbbjBrzBODcsAQ,qpub9UXk8oBJr95oq-MbIg,DkZuVzYuylBanEsWK7mNGw,Q7m8G4579-XYKV3LMW8V7g,xfzDtGxx6B4a27RAyah69w,...,5JNvfPfd6mcpwNoekzqBow,TFybLreUjQyx0zLq5tW7VA,fWEfX14W2_7GDOJBSQQU9Q,4Azs1HUqi0Nk6s6xB_9jsQ,oljOuU-B7XUNQFPBjed2yQ,DgCAM01n2Qo5DsoKj_zkug,3sPSc8NBvrkXi7scpRYDLg,rdZ6xVABSoOVeAIiCaEj4w,ATErtjbLCxEPOD8ICKKr5Q,q3IEuy9YPsHh0hpqX1HJZw
5102,zwNC-Ow4eIMan2__bS9-rg,2XkVexeY_8zyc9nriSElQg,JIfSollg32ifUzTw2K4PDQ,4mb32UmQULqg7IMck28vog,Q4AWWdBWe_ojYX1a4ffXFQ,nt2-Zk4FmGY2SYSDBI0gHw,CVECSCW-7zDzjpIPxsew4g,QF2P3oJqCJC9z0TPDDFVxw,R7LyTeiOHLyTNkA8HssBRQ,Gud0G5LL9vFOOHjvyEIlTQ,...,6Ve_2kEz18dvhp4q3M5I2Q,1mSAHgRm_sbDslTgeId2Mg,HnQVDNGFWOS5zle0dhpOkw,iX-I9C9P6xFNjqrnpAU73w,YcGHCoSkl2PLKsHQCiu0HA,6e0Abng1LRexKI4LRJTX_A,d4qwVw4PcN-_2mK2o1Ro1g,SzB85-8tmDEvoapZSDTBkQ,8hZjiPzJIojA1k7_W4hELA,gTHo2aGgiG-jqy-_C2OhSg
5103,zwmKWhyyV-zejEUbQzBtHw,Yqgyx8SJ5SqFYc-4yH6Z1g,YTfXZ9tbTBJkWsDI7cMDWQ,slUn4AINGy3QmDK62YGzoA,u3nwislXMKKwkpHAXAArbQ,310Eyru1zsZ9K7BA-bgPQw,F06m2yQSPHIrb1IT7heYeQ,jfmyj519pg2flxGEQ2xdPg,M9zu5KTyVwVC-45Mn8Wp0g,3TRgD3KLVis61eGvMZrykA,...,KhvwEXKJXRTgcJRNmJNhsA,Oyo5CWthu2upxxaf69MfIg,4JkGjRRc3XucG9U8XwthGg,CVRTly4ag_q5ua0R3iqrVw,DIxgItsSI9QwX9H8lVptYg,0eZHVYDyACtVVOIgoBlV3g,1qTx9z8RMz7RIYKAfnpZ0A,X5uxdU9GHoUAqo2wmbg3OQ,UMWzaFhW6M3nsIDyIpMfhg,UDE4VL0k8Zrko7RkJ4agGQ
5104,zx_j6OuuHHa2afVoAZuLpA,iX2c4I

In [69]:
result_df.to_csv('LV_top_100_rest.csv')

### Other Try

In [ ]:
for i in tqdm(range(len(cities))):
    business_city = restaurants_business[restaurants_business.city == cities[i]]
    bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
def calc_sim_user_rating(target_user_id, target_business_id, groupby_df, original_df):
    business_id_df = original_df[original_df.business_id == target_business_id].copy()
    
    target_user_tfidf = tfidf_matrix[groupby_df[groupby_df.user_id == target_user_id].index[0]].copy()
    
    dist_user = []
    for user_id in business_id_df.user_id.values:
        dist_user.append(cosine_similarity(tfidf_matrix[groupby_df[groupby_df.user_id==user_id].index[0]],
                                           target_user_vector)[0][0])
        
    business_id_df['similarity'] = dist_user    
    #Filtering technique = Calc percentile and filter percentiles > 80%
    business_id_df = business_id_df[business_id_df.rank(pct=True).similarity > .8]
     
    rating_diff = []
    for user_id in business_id_df.user_id.values:
        rating_diff.append(test_rest_user_id.stars_review[test_rest_user_id.user_id==user_id].values[0]
                            -np.average(city_train[city_train.user_id ==user_id].stars_review.values))
    test_rest_user_id['rating_diff'] = rating_diff
    
    return np.average(test_rest_user_id.rating_diff)

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in test.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], test['user_id'][i]) for i in similar_indices]

    results[row['user_id']] = similar_items[1:]
    
print('done!')

def item(id):
    return test.loc[test['user_id'] == id]['text'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")